## Test NIRSpec IFS data in Cubeviz

We will open a NIRSpec simulated cube in Cubeviz. The header is not compliant with a standard JWST pipeline product, but we can create a Spectrum1D object and load that instead of the fits file directly. We will then create a flux map of one of the lines in the spectra.

**Installation instructions**  
- conda create -n jdaviz python=3.8
- conda activate jdaviz
- pip install git+https://github.com/spacetelescope/jdaviz.git

In [ ]:
#to use 100% of the browser window
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import time

import specutils
from specutils import Spectrum1D

import jdaviz
from jdaviz import CubeViz

import astropy
from astropy.io import fits
from astropy import units as u
from astropy.nddata import StdDevUncertainty
import astropy.wcs

import numpy as np

Check versions. Works with:
- jdaviz: 2.0.1.dev92+g26563fd
- astropy: 4.3.1
- specutils: 1.4.2.dev42+g08b2a7f3

In [ ]:
print("jdaviz:",jdaviz.__version__)
print("astropy:",astropy.__version__)
print("specutils:",specutils.__version__)

## Load a pipeline-processed JWST simulated cube as example

In [ ]:
from astropy.utils.data import download_file
filename = "https://stsci.box.com/shared/static/ff0bj31acot1272x5qq2clbmto3s6v4f.fits"
ifucube_file = download_file(filename, cache=True)
#cubeviz.load_data(fn)

In [ ]:
cubeviz2 = CubeViz()
cubeviz2.app

In [ ]:
cubeviz2.app.load_data(ifucube_file)

## Explore the IPS file as is

In [ ]:
filename = "./3004_IPS_18660_badrc_CUBE_R3D.fits"

In [ ]:
hdulist = fits.open(filename)
hdulist.info()

In [ ]:
fnhead_sci = hdulist['DATA'].header
fnhead_err = hdulist['ERR'].header
fnhead_dq = hdulist['QUALITY'].header
hdulist['DATA'].header

## Create Spectrum1D object

In [ ]:
# Check the wavelength array
wave = np.arange(fnhead_sci['CRVAL3'], fnhead_sci['CRVAL3'] + float(fnhead_sci['NAXIS3'])*fnhead_sci['CDELT3'], fnhead_sci['CDELT3']) * 1.E6 * u.um
print(wave)
print(len(wave))
print(fnhead_sci['NAXIS3'])

In [ ]:
# Create a WCS object instead
wcs_dict = {
'CTYPE1': 'WAVE    ', 'CUNIT1': 'um', 'CDELT1': fnhead_sci['CDELT3'] * 1E6, 'CRPIX1': fnhead_sci['CRPIX3'], 'CRVAL1': fnhead_sci['CRVAL3'] * 1E6, 'NAXIS1': fnhead_sci['NAXIS3'],
'CTYPE2': 'HPLT-TAN', 'CUNIT2': 'deg', 'CDELT2': fnhead_sci['CDELT1'], 'CRPIX2': fnhead_sci['CRPIX1'], 'CRVAL2': fnhead_sci['CRVAL1'], 'NAXIS2': fnhead_sci['NAXIS1'],
'CTYPE3': 'HPLN-TAN', 'CUNIT3': 'deg', 'CDELT3': fnhead_sci['CDELT2'], 'CRPIX3': fnhead_sci['CRPIX2'], 'CRVAL3': fnhead_sci['CRVAL2'], 'NAXIS3': fnhead_sci['NAXIS2']}
input_wcs = astropy.wcs.WCS(wcs_dict)

In [ ]:
#rearrange the data
data = hdulist['DATA'].data
print(data.shape)

new_data = np.zeros((35,30,3817))
for i in range (0,35):
    for j in range (0,30):
        for k in range (0,3817):
            new_data[i,j,k] = data[k,i,j]

print(new_data.shape)


In [ ]:
#full cube with wcs
cube = Spectrum1D(flux=new_data * u.uJy,
                  wcs=input_wcs,
                  uncertainty = StdDevUncertainty(new_data*0.01*u.uJy))

## Load the spectrum in Cubeviz

In [ ]:
cubeviz = CubeViz()
cubeviz.app

In [ ]:
# This opens the full cube in the form of a Spectrum1D object
cubeviz.app.load_data(cube, data_label="cube")

## Work in the GUI

### Adjust the settings to properly see the cube
- In the image viewer on the top left, use the second icon to the right
- Select the percentile to 95%
- Scroll though the cube to have an idea what you are looking at

### Get a useful 1D spectrum
- In the cube viewer, click the first icon to the right and select the circle
- Draw a region in the center of the field of view to select the part of the cube with the better signal
- In the spectrum viewer, the dafault spectrum plotted is the spectrum of the brightest pixel
- Choose Median using the second icon to the right and the menu "Viewer"
- Click the first icon to the right and the home icon that appears to reset the zoom to a more appropriate value
- You should be able to find the emission lines!

### Select the spectral region of Ha
- In the spectrum viewer, use the first icon to the right and select the region icon
- Select the spectrum around Ha

### Calculate the moment map
- Open the plugin tray by clicking the icon at the top right
- Select the moment map plugin
- Select the appropriate subset as spectral region
- Select moment "0" and hit Calculate

### View the moment map in the viewer
- In one of the empty viewer, click Data
- Select the new moment map
- Change the custom percentiles to better see the flux map of the line


## Get the moment map programmatically

In [ ]:
cubeviz.app.get_data_from_viewer("uncert-viewer")

In [ ]:
moment_map_cube = cubeviz.app.get_data_from_viewer("uncert-viewer", "Moment 0: Unknown spectrum object[FLUX]")
hdulist = moment_map_cube.to_hdu()
moment_map_cube.write('moment0_cube.fits')

## Visualize the line map in Imviz

In [ ]:
from jdaviz import Imviz

In [ ]:
imviz = Imviz()
imviz.app

In [ ]:
imviz.load_data('moment0_cube.fits')